In [ ]:
class Field:
    def __init__(self, option='classic'):
        self.convert_dict = {'A': '1',
                             'B': '2',
                             'C': '3',
                             'D': '4',
                             'E': '5',
                             'F': '6',
                             'G': '7',
                             'H': '8'}
        if isinstance(option, str):
            self.option = option
        else:
            self.option = 'classic'
        self.create_empty_board()
        self.expose()
        self.player_generator = self.generate_player_name()
        self.player = None

    def create_empty_board(self):
        self.board = [[i for i in '*ABCDEFGH']]
        for j in range(8):
            s = [j + 1]
            for i in range(8):
                s.append(Empty('ate', [i, j]))
            self.board.append(s)

    def expose(self):
        for i in range(8):
            self.board[2][i + 1] = Pawn('white', [2, i + 1])
            self.board[7][i + 1] = Pawn('black', [2, i + 1])
        c, line = 'white', 1 #
        self.board[line] = [line, Rook(c, [line, 1]), Bishop(c, [line, 2]), Knight(c, [line, 3]), Queen(c, [line, 4]),
                            King(c, [line, 5]), Knight(c, [line, 7]), Bishop(c, [line, 6]), Rook(c, [line, 8])]
        c, line = 'black', 8
        self.board[line] = [line, Rook(c, [line, 1]), Bishop(c, [line, 2]), Knight(c, [line, 3]), Queen(c, [line, 4]),
                            King(c, [line, 5]),
                            Knight(c, [line, 7]), Bishop(c, [line, 6]), Rook(c, [line, 8])]
        self.board[8][4] = Queen('black', [8, 4])
        self.board[8][5] = King('black', [8, 5])
        self.board[1][4] = Queen('white', [1, 4]) #
        self.board[1][5] = King('white', [1, 5]) #
        self.show_board()

    def generate_player_name(self):
        while True:
            for i in 'white', 'black':
                yield i

    def play(self):
        moves = ['a1-h1']
        while not self.check_win():
            self.player = next(self.player_generator)
            move = input(f'Ход {self.player} >> ')
            # move = 'a1-h1'
            move = self.convert(move)
            while not (
                    self.input_validate(move) and self.shape_validate(move.split('-')[0]) and self.check_cords(move)):
                if not (self.input_validate(move)):
                    move = self.convert(input('Некорректный ввод\n>> '))
                elif not self.shape_validate(move.split('-')[0]):
                    move = self.convert(input(f'Вы не можете походить этой фигурой\n>> '))
                elif not (self.check_cords(move)):
                    move = self.convert(input('Так ходить нельзя!\n>> '))
                else:
                    move = input(f'Произошла неизвестная ошибка. Попробуйте снова. Если проблема не решится, то обра'
                                 f'титесь в службу поддержки или перезапустите игру!\n>> ')
                    move = self.convert(move)
            moves.append(move)
            self.move(move)
            self.show_board()

    def check_turn_for_possible(self):
        pass


    def input_validate(self, move):
        if len(move) != 5:
            return False
        return True

    def convert(self, move):
        move = move.upper()
        for i in range(len(move)):
            if move[i] in self.convert_dict.keys():
                move = move[:i] + self.convert_dict[move[i]] + move[i + 1:]
        return move

    def shape_validate(self, coords):
        if self.board[int(coords[1])][int(coords[0])].color == self.player:
            return True
        return False

    def check_win(self):
        return False

    def show_board(self):
        print('※'.join(self.board[0]))
        for line in self.board[1:]:
            print(*line)
        print("-" * 15)

    def switch(self, pos1, pos2):
        if str(self.board[pos2[0]][pos2[1]]) == '※':
            trash = self.board[pos1[0]][pos1[1]]
            self.board[pos1[0]][pos1[1]] = self.board[pos2[0]][pos2[1]]
            self.board[pos2[0]][pos2[1]] = trash
        else:
            self.board[pos2[0]][pos2[1]].color = 'ate'
            trash = self.board[pos1[0]][pos1[1]]
            self.board[pos1[0]][pos1[1]] = self.board[pos2[0]][pos2[1]]
            self.board[pos2[0]][pos2[1]] = trash

    def check_cords(self, input):
        input = input.split('-')
        return (self.board[int(input[0][1])][int(input[0][0])].check_move([int(input[1][1]), int(input[1][0])]))

    def move(self, input):
        input = input.split('-')
        self.switch([int(input[0][1]), int(input[0][0])], [int(input[1][1]), int(input[1][0])])


class Figure(object):
    def __init__(self, color, position: list):
        self.color = color
        self.position = position

    def update_position(self, new_position):
        self.position = new_position


class Empty(Figure):
    def __str__(self):
        return '※'


class Pawn(Figure):
    def __str__(self):
        if self.color == 'white':
            return '♟'
        elif self.color == 'black':
            return '♙'
        elif self.color == 'ate':
            return '※'

    def check_move(self, new_position):
        if self.position[1] == new_position[1] and self.position[0] - new_position[0] < 2:
            self.update_position(new_position)
            return True
        elif abs(self.position[0] - new_position[0]) == 1 and abs(self.position[1] - new_position[1]) == 1:
            return True
        return False


class Rook(Figure):
    def __str__(self):
        if self.color == 'white':
            return '♜'
        elif self.color == 'black':
            return '♖'
        elif self.color == 'ate':
            return '※'

    def check_move(self, new_position):
        return (self.position[0] == new_position[0]) or (self.position[1] == new_position[1])


class Bishop(Figure):
    def __str__(self):
        if self.color == 'white':
            return '♝'
        elif self.color == 'black':
            return '♗'
        elif self.color == 'ate':
            return '※'

    def check_move(self, new_position):
        return abs(self.position[0] - new_position[0]) == abs(self.position[1] - new_position[1])


class Knight(Figure):
    def __str__(self):
        if self.color == 'white':
            return '♞'
        elif self.color == 'black':
            return '♘'
        elif self.color == 'ate':
            return '※'

    def check_move(self, new_position):
        deltax = abs(self.position[0] - new_position[0])
        deltay = abs(self.position[1] - new_position[1])
        return (deltax == 2 and deltay == 1) or (deltay == 2 and deltax == 1)


class King(Figure):
    def __str__(self):
        if self.color == 'white':
            return '♚'
        elif self.color == 'black':
            return '♔'
        elif self.color == 'ate':
            return '※'

    def check_move(self, new_position):
        deltax = abs(self.position[0] - new_position[0])
        deltay = abs(self.position[1] - new_position[1])
        return (deltay <= 1) and (deltax <= 1) and (deltay + deltax) != 0


class Queen(Figure):
    def __str__(self):
        if self.color == 'white':
            return '♛'
        elif self.color == 'black':
            return '♕'
        elif self.color == 'ate':
            return '※'

    def check_move(self, new_position):
        return (self.position[0] == new_position[0]) or (self.position[1] == new_position[1]) or abs(
            self.position[0] - new_position[0]) == abs(self.position[1] - new_position[1])


if __name__ == '__main__':
    game = Field()
    game.play()


*※A※B※C※D※E※F※G※H
1 ♜ ♝ ♞ ♛ ♚ ♞ ♝ ♜
2 ♟ ♟ ♟ ♟ ♟ ♟ ♟ ♟
3 ※ ※ ※ ※ ※ ※ ※ ※
4 ※ ※ ※ ※ ※ ※ ※ ※
5 ※ ※ ※ ※ ※ ※ ※ ※
6 ※ ※ ※ ※ ※ ※ ※ ※
7 ♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙
8 ♖ ♗ ♘ ♕ ♔ ♘ ♗ ♖
---------------
Ход white >> a1-a4
*※A※B※C※D※E※F※G※H
1 ※ ♝ ♞ ♛ ♚ ♞ ♝ ♜
2 ♟ ♟ ♟ ♟ ♟ ♟ ♟ ♟
3 ※ ※ ※ ※ ※ ※ ※ ※
4 ♜ ※ ※ ※ ※ ※ ※ ※
5 ※ ※ ※ ※ ※ ※ ※ ※
6 ※ ※ ※ ※ ※ ※ ※ ※
7 ♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙
8 ♖ ♗ ♘ ♕ ♔ ♘ ♗ ♖
---------------
